In [264]:
from dotenv import load_dotenv 

load_dotenv('../../.env')

True

In [265]:
from langchain.tools import DuckDuckGoSearchRun
from langchain_experimental.tools import PythonREPLTool

tools = [DuckDuckGoSearchRun()]

In [266]:
from langgraph.prebuilt import ToolExecutor

tool_executor = ToolExecutor(tools)

In [267]:
from langchain_openai import ChatOpenAI

# We will set streaming=True so that we can stream tokens
# See the streaming section for more information on this.
model = ChatOpenAI(temperature=0, streaming=True)

In [268]:
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain import hub

prompt = hub.pull("hwchase17/openai-tools-agent")

agent = create_openai_tools_agent(llm=model, tools=tools, prompt=prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools)

functions = [format_tool_to_openai_function(t) for t in tools]
model = model.bind_functions(functions)

## Define the nodes

We now need to define a few different nodes in our graph.
In `langgraph`, a node can be either a function or a [runnable](https://python.langchain.com/docs/expression_language/).
There are two main nodes we need for this:

1. The agent: responsible for deciding what (if any) actions to take.
2. A function to invoke tools: if the agent decides to take an action, this node will then execute that action.

We will also need to define some edges.
Some of these edges may be conditional.
The reason they are conditional is that based on the output of a node, one of several paths may be taken.
The path that is taken is not known until that node is run (the LLM decides).

1. Conditional Edge: after the agent is called, we should either:
   a. If the agent said to take an action, then the function to invoke tools should be called
   b. If the agent said that it was finished, then it should finish
2. Normal Edge: after the tools are invoked, it should always go back to the agent to decide what to do next

Let's define the nodes, as well as a function to decide how what conditional edge to take.

**STREAMING**

We define each node as an async function.

In [269]:
from langgraph.prebuilt import ToolInvocation
import json
from langchain_core.messages import FunctionMessage, AIMessage, HumanMessage
from typing import Annotated, Sequence, TypedDict
import operator
from langchain_core.messages import BaseMessage



class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    # next: str


def should_continue(messages):
    # messages = state['messages']
    print('\n', messages)
    last_message = messages[-1]
    # If there is no function call, then we finish
    if 'additional_kwargs' not in last_message or "function_call" not in last_message.additional_kwargs:
    # if "function_call" not in last_message.additional_kwargs:
        return "end"
    # Otherwise if there is, we continue
    else:
        return "continue"


async def call_model(messages):
    # messages = state['messages']
    # response = await model.ainvoke(messages)
    response = await agent_executor.ainvoke({
        'input': messages[-1].content, 
        'chat_history': messages[:-1]}
    )
    
    # We return a list, because this will get added to the existing list
    return HumanMessage(content=response['output'])


async def call_tool(messages):
    # messages = state['messages']
    last_message = messages[-1]

    action = ToolInvocation(
        tool=last_message.additional_kwargs["function_call"]["name"],
        tool_input=json.loads(
            last_message.additional_kwargs["function_call"]["arguments"]
        ),
    )
    # We call the tool_executor and get back a response
    response = await tool_executor.ainvoke(action)
    # We use the response to create a FunctionMessage
    function_message = FunctionMessage(content=str(response), name=action.tool)
    # We return a list, because this will get added to the existing list
    return function_message

## Define the graph

We can now put it all together and define the graph!

In [270]:
from langgraph.graph import MessageGraph, END

workflow = MessageGraph()

workflow.add_node("agent", call_model)
workflow.add_node("action", call_tool)

workflow.set_entry_point("agent")

workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue": "action",
        "end": END,
    },
)

workflow.add_edge("action", "agent")

app = workflow.compile()

## Streaming LLM Tokens

You can access the LLM tokens as they are produced by each node. 
In this case only the "agent" node produces LLM tokens.
In order for this to work properly, you must be using an LLM that supports streaming as well as have set it when constructing the LLM (e.g. `ChatOpenAI(model="gpt-3.5-turbo-1106", streaming=True)`)


In [271]:
from langchain_core.messages import HumanMessage

inputs = [HumanMessage(content="what is the weather in Oslo?")]
# inputs = [{'role': 'human', 'content': "what is the weather in Oslo?"}]
async for event in app.astream_events(inputs, version="v1"):
# async for event in app.astream_events({'messages': inputs}, version="v1"):
# async for event in app.astream_events('Hvordan er været i Trondheim?', version='v1'):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

TypeError: list indices must be integers or slices, not str